# Inundation History Tool

## Purpose


This tool is used for associating river flow rates with Landsat and Sentinel satellite passes which are accessed through [Google Earth Engine](https://developers.google.com/earth-engine). It filters satellite passes within defined flow bands of interest, removes cloudy imagery, and also applies a filter to those images on the rising/falling limb. The images are then indexed using the MNDWI, NDWI, Fisher index and a custom False colour (swir2, nir and red) index for water identification.

The tool outputs the images as Geotiff files to Google Cloud, Google Drive or as an Asset for use in GIS software (see [Exporting Data](https://developers.google.com/earth-engine/guides/exporting) user guide).

> **Note:** To use this notebook you will need the have a Google Earth Engine account, subject to their [Terms and Conditons](https://earthengine.google.com/terms/)

This tool has been designed to run in Jupyter Notebooks.

## Quick use notes

For **issues relating to the script, a tutorial, or feedback** please contact Martin Job at martin.job@mdba.gov.au, Gabrielle Hunt at gabrielle.hunt@mdba.gov.au or Beau Byers at beau.byers@mdba.gov.au:
1. Authenticate to GEE. Right-click on the link to open in a new tab, sign in, copy the code, paste it into the box and press 'enter' only.
2. Press shift + enter on following cells until IHT Dashboard appears. Use the dashboard to input your selections. Start by selecting the Jupyter environment you are deploying the tool in, then a date range of interest, then defining the flow band of interest. You then have three options (select one) for defining the extents of your location of interest. Extents can be defined by either uploading a shapefile, typing in the coordinates manually, or using the coordinates of the gauge you selected in the previous step. You can then type in a gauge number and select if you want to include landsat 7 passes with the SLC failure (leading to stripes in these images)
3. Press shift + enter until you get to 'PART 2'. Here you can toggle the satellite passes you wish to analyse and export based on their position on the hydrograph.
5. Open your google cloud storage and download the Geotiff files for import into your preferred GIS software.
6. OPTIONAL: download any of the plots of interest.
7. OPTIONAL: Download the csv, containing the date of the satellite flyover, gauge reading, and whether you considered it a pass or fail.
8. Once you have finished, restart the kernal to remove the variables and rerun the cells with your next parameters

## Part 1: Load the packages and set up Earth Engine

### 1.1 Load modules

When running the IHT foor the first time you may need to run the following cells to establish the environment. Uncomment the required cells.

If running in Colab run the following cell.

In [ ]:
# pip install jupyterlab ipywidgets geopandas ipyleaflet

Import packages

In [ ]:
# Importing the MDBA gauge getter:

!git clone https://github.com/MDBAuth/MDBA_Gauge_Getter.git
!cd MDBA_Gauge_Getter; python setup.py install

In [ ]:
import os
import re
import sys
import warnings
from datetime import datetime
from dateutil.parser import parse
from pathlib import Path
import numpy as np
import ee
import pandas as pd
import xarray as xr
import geopandas as gpd
import plotly.offline as py
import plotly.graph_objects as go

import ipywidgets as widgets
import folium

from MDBA_Gauge_Getter.mdba_gauge_getter import gauge_getter as gg

# Local modules for IHT:
import iht
import iht_functions

### 1.2 Authenticate and Initialise Google Earth Engine
You may need to open the link in a new tab. Sign into you Google Earth Engine account and copy the code into the box. Then press `enter` (not `shift + enter`)

In [ ]:
ee.Authenticate()
ee.Initialize()

## Part 2: Set your parameters

Run this cell and a dashboard will appear for you to interact with.

1. Deployment environment (i.e. where is Jupyter running?)
2. Start Date = Landsat imagery available from Oct 1987, Sentinel from Jul 2015
3. Minimum flow = a value below 100 ML/day may result in the tool crashing due to too many images to process
   Maximum flow = any value can be used
4. Buffer (default 0.2) = buffer applied to selected area
5. One of the options must be selected.
   If the shapefile option is selected, all shapefile files must be loaded into the 'shapefile_inputs' folder within the IHT directory for the tool to access. WGS84 shapefile errors have been encountered, may require reprojection to GDA1994.
6. Gauge number = can be found through the BoM (http://www.bom.gov.au/waterdata/) or by using the optional gauge map below the dashboard
7. SLC failure = a sensor failure in Landsat 7 resulted in limited data collection after 21 May 2003, images may be useful for limited applications and are generally excluded
8. Cloud threshold (default 5) = percentage of clouds in the image that the user belives is acceptable for your use, 0 would mean that only images with 0% cloud presence will be used, 100 would mean that images with up to 100% cloud cover will be used.
9. Export variables = the users preference for different water indexes or RGB imagery which is exported in the final processes of the IHT.
   Threshold for summary images (default 0.1) = MNDWI threshold for water detection, can be adjusted from -1 to 1, where all pixels above the threshold will be classified as water for calculation of % inundation
   
After selecting your variables, run the next cell to load them from the dashboard into the notebook.

The final cell will allow you to check your area of interest to confirm that the spatial extent selected correctly matches with the users requirements.

### 2.1 Load the dashboard and save them to the notebook:

In [ ]:
iht.iht_dashboard

Once you have finished making your selections, run the below cell to commit them to the notebook

In [ ]:
# Get IHT variables
iht_values = iht.IhtValues()
if not iht_values.valid:
    for i in iht_values.errors:
        print(i)
    raise Exception("IHT input form errors exist. Please correct the above errors.")

flow_band = {
    'min flow': iht_values.min_flow, 
    'max flow': iht_values.max_flow
}

# Pull gauge data then clean the gauge data into workable format:
gauge_data = gg.gauge_pull([iht_values.gauge_num.replace(' ', '')],
                                          iht_values.start_date, 
                                          iht_values.end_date)

# Define an area of interest from the IHT results
lat_low, lat_high, lon_low, lon_high = iht.get_coords()

aoi = ee.Geometry.Polygon([[
                            [lon_low, lat_low],
                            [lon_low, lat_high],
                            [lon_high, lat_high],
                            [lon_high, lat_low]
                          ]], None, False)
# Get user request:
define_gauge = iht_values.check_gauge
define_coords = iht_values.check_own
define_shapefile = iht_values.check_shapefile

shapefile_name = iht_values.shapefile_loc

# Mount Google Drive
if iht_values.deploy_environment == 'Google colab':
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('drive/MyDrive/IHT')
else:
    pass

### Optional Gauge number map

If you require assistance to find a suitable gauge number, run the optional cell below which will load an interactive map with gauge numbers, if you do not require it then skip running it.

In [ ]:
# Import gauge locations
data = pd.read_csv('gauge_data/bom_gauge_data.csv')

# Create a map
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    try:
        folium.CircleMarker(location=[point.lat, point.lon],
                            radius=2,
                            weight=5,
                            popup=str(point['gauge number'])
                           ).add_to(this_map)
    except:
        pass
    
# Use df.apply(,axis=1) to "iterate" through every row in your dataframe
data.apply(plotDot, axis = 1)

# Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

this_map

### 2.2 Confirm your area of interest:
The cell below will create a custom map, and a red square will display where your area of interest (aoi) is based on your selections and inputs to the IHT Dashboard. If the aoi does not display, the tool does not have a spatial extent and will not be able to process your exports. The map may take some time to display.

In [ ]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = iht_functions.add_ee_layer

# Create a folium map object.
m = folium.Map(location=[-32, 145], zoom_start=5, height=500)
vis_params = {'palette':['#B22222'], 'opacity': 0.3}
m.add_ee_layer(ee.Image(1).clip(aoi), vis_params, 'aoi')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

print('Your area of interest is show in red, click the checkbox to turn it off.')
display(m)

## Part 3: Loading and processing satellite imagery

### 3.1 Load GEE imagery

To load imagery from GEE generally you will need to:
- define an area of interest and pass to `filterBounds`
- define a time period of interest and pass to `filterDate`
- filter on metadata (commonly cloud cover)
- select bands of interest

>**Note**: All options for how to manipulate GEE objects can be found in Google Earth Engine's [User Guides](https://developers.google.com/earth-engine/apidocs)

In [ ]:
# Change dates to consistent datetime type to allow for comparison:
slcError = datetime.strptime('2003-05-01','%Y-%m-%d').date()
todays_date_str = datetime.today().strftime('%Y-%m-%d')

start_date = iht_values.start_date
end_date = iht_values.end_date

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

if iht_values.slc_option == 'Exclude':
    end_time_ls7 = min(end_date, slcError).strftime('%Y-%m-%d')
else:
    end_time_ls7 = end_date_str

# Filter landsat and sentinel collections to get desired image
ls8_imcol = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
              .filterBounds(aoi) \
              .filterDate('2013-04-01', todays_date_str) \
              .filter(ee.Filter.lt('CLOUD_COVER',iht_values.cloud_threshold)) \
              .map(iht_functions.maskQuality)\
              .map(iht_functions.bands8)

ls7_imcol = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
              .filterBounds(aoi) \
              .filterDate('1999-01-01',end_time_ls7) \
              .filter(ee.Filter.lt('CLOUD_COVER',iht_values.cloud_threshold)) \
              .map(iht_functions.maskQuality) \
              .map(iht_functions.bands57)

ls5_imcol = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
              .filterBounds(aoi) \
              .filterDate('1984-03-01','2012-06-01') \
              .filter(ee.Filter.lt('CLOUD_COVER',iht_values.cloud_threshold)) \
              .map(iht_functions.maskQuality) \
              .map(iht_functions.bands57)

ls_imcol = ls8_imcol.merge(ls7_imcol.merge(ls5_imcol)) \
                    .filterDate(start_date_str, end_date_str) \
                    .map(iht_functions.clipEdges)

s_imcol = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterBounds(aoi) \
            .filterDate(start_date_str, end_date_str) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',iht_values.cloud_threshold)) \
            .map(iht_functions.maskS2clouds) \
            .map(iht_functions.bandsS2)

In [ ]:
# Reformat the satellite times
ls_times = iht_functions.getTime(ls_imcol)
s_times = iht_functions.getTime(s_imcol)

### 3.2 Filter satellite imagery and merge with gauge data

In [ ]:
# Clean the gauge data
clean_gauge_data = iht_functions.gauge_data_cleaner(gauge_data)

The following cell will print a summary of the valid satellite passes that are available based on your preferences, note that more than 400 passes may cause the tool to crash. If the number of satelllite passes exceeds 400, consider constraining your variables in the IHT Dashboard (narrow date range, minimum and maximum flow thresholds or cloud threshold), and run all cells from the IHT Dashboard down.

In [ ]:
#Check how many passes you are about to load. 
#Loading over 400 passes may cause the kernal to crash
print('Landsat has ' + iht_functions.countPasses(ls_times) + ' valid passes.')
print('Sentinel has ' + iht_functions.countPasses(s_times) + ' valid passes.')

In [ ]:
# Merge satellite data with gauge data
ls_count, ls_merged_data = iht_functions.merge_satellite_with_gauge(ls_times, clean_gauge_data, flow_band)  
s_count, s_merged_data = iht_functions.merge_satellite_with_gauge(s_times, clean_gauge_data, flow_band)

In [ ]:
# Convert to a pandas dataframe    
ls_all_merged_data = iht_functions.convert_to_pandas(ls_merged_data, ls_times)
s_all_merged_data = iht_functions.convert_to_pandas(s_merged_data, s_times)

### 3.3 Graph all avaliable imagery
This cell will create an interactive graph which will show you the Landsat and Sentinel images that are available based on your variables.

In [ ]:
# Generate graph showing the satellite passes relative to where they occur on the hydrograph
fig_all = iht_functions.graph_all(flow_band, clean_gauge_data, ls_all_merged_data, s_all_merged_data)
fig_all

## Part 4: Separate passes into rising and falling categories based on gauge data

This box will check whether the gauge-reading 21 days (`days_ahead`) after the satellite pass was higher or lower than the day of the satellite pass. The multiplier represents by how much more the water should be lower or higher to be considered a significant change. This (`multiplier`) has been defined as 1 for simplicity. It puts the pass either into the rising or falling list accordingly. It runs a loop to do this for every single pass. The output will tell you how many passes you got in each list and show you how the passes were catagorised on a hydrogaph. A manual reclassification is available later in the IHT to rectify any misclassifications.

> **Note:** Check the graph in the output to see how well the data was separated. Consider changing `days_ahead` or `multiplier` in the code block below.

### 4.1 Seperate passes into rising and falling

In [ ]:
# Use custom functions to seperate into rising and falling

multiplier = 1 # how much more the flow rate has to rise by to be considered a significant rise
days_ahead = 21 # how many days in advance the algorithm checks for a rise or fall

ls_rising, ls_falling = iht_functions.rising_falling_main(multiplier, days_ahead, clean_gauge_data, ls_all_merged_data)
s_rising, s_falling = iht_functions.rising_falling_main(multiplier, days_ahead, clean_gauge_data, s_all_merged_data)

### 4.2 Graph the results

In [ ]:
graph_rising_falling = iht_functions.graph_rising_falling(flow_band, clean_gauge_data, ls_rising, ls_falling, s_rising, s_falling)
graph_rising_falling

# PART 5 - Decide which passes to include in the output

The program has split the satellite passes into the rising and falling limb and allocated them to a 'pass' and 'fail' category respectively. Only the satellite passes in the 'pass' category will be analysed. View the graph below, and click on the satellite passes you would like to reclassify, and the program will reclassify them for you (depending on your deployment environment this feature may not be available to you; if this is the case, you will be able to reclassify using a dropdown list).

### 5.1 Determine what passes will be accepted or rejected

In [ ]:
ls_rising_clean = iht_functions.rising_falling_cleaner(ls_rising, 'landsat', 'accept')
ls_falling_clean = iht_functions.rising_falling_cleaner(ls_falling, 'landsat', 'reject')
s_rising_clean = iht_functions.rising_falling_cleaner(s_rising, 'sentinel', 'accept')
s_falling_clean = iht_functions.rising_falling_cleaner(s_falling, 'sentinel', 'reject')
    
master_df = pd.concat([ls_rising_clean, ls_falling_clean, s_rising_clean, s_falling_clean])

# Concatenate the landsats to the one dataframe, handle case where a dataframe could be empty:
if ls_rising_clean is not None and ls_falling_clean is not None:
    ls_only = pd.concat([ls_rising_clean, ls_falling_clean])
elif ls_rising_clean is not None:
    ls_only = ls_rising_clean.copy(deep=True)
elif ls_falling_clean is not None:
    ls_only = ls_falling_clean.copy(deep=True)
    
# Concatenate the landsats to the one dataframe, handle case where a dataframe could be empty:
if s_rising_clean is not None and s_falling_clean is not None:
    s_only = pd.concat([s_rising_clean, s_falling_clean])
elif s_rising_clean is not None:
    s_only = s_rising_clean.copy(deep=True)
elif s_falling_clean is not None:
    s_only = s_falling_clean.copy(deep=True)

master_df_reindex = master_df.reset_index()

In [ ]:
# # Alternatively to accept all passes:
# master_df_reindex = master_df_reindex.replace('sentinel reject','sentinel accept')
# master_df_reindex = master_df_reindex.replace('landsat reject','landsat accept')

### 5.2 Graph accepted and rejected passes
> **NOTE:** CLICK THE SATELLITE POINTS ON THE GRAPH TO CHANGE CATEGORY BETWEEN PASS AND FAIL AND VICE VERSA

If running in Google Colab or AZML the graph will not be interactive. Use the date dropdown and redraw the graph to display changes. 

In [ ]:
google_colab_or_AZML = iht_values.deploy_environment == 'Google colab' or iht_values.deploy_environment == 'AZML'

In [ ]:
if google_colab_or_AZML:
    fig = iht_functions.addAcceptRejectGraph(s_only, ls_only, master_df_reindex,clean_gauge_data,flow_band)
    fig.show()
else:
    display(iht_functions.addAcceptRejectGraph(s_only, ls_only, master_df_reindex,clean_gauge_data,flow_band))
    #TODO why is this not displaying

In [ ]:
if google_colab_or_AZML:
    print('Use Crtl to select mutliple dates to switch from "accept" to "reject" and vice versa')
    df = master_df_reindex
    datelist  = sorted(df['date'].to_list())
    widget_selection = widgets.SelectMultiple(
        options=datelist,
        disabled=False
    )
    display(widget_selection)
else:
    pass

In [ ]:
if google_colab_or_AZML:
    # Get list of dates from widget
    select_list = list(widget_selection.value)

    # Select rows based on dates
    df[df['date'].isin(select_list)]

    # Add change column
    df['Change'] = df['date'].isin(select_list)

    master_df_reindex = df.apply(iht_functions.switch, axis=1)

    fig = iht_functions.addAcceptRejectGraph(s_only, ls_only, master_df_reindex,clean_gauge_data,flow_band)
    fig.show()
else:
    pass

## Part 6: Export imagery to Google Cloud
Images can also be exported to Drive or as an Asset (see [Exporting Data](https://developers.google.com/earth-engine/guides/exporting) user guide). For large amounts of data you can download these to your local machine using [gsutil](https://cloud.google.com/storage/docs/gsutil) in the Google Cloud SDK. For small amounts the can be manually downloaded directly from Google Could or Google Drive.  

In the following section you:
- Calculate indices:
        - NDWI    
        - MNDWI
        - Fisher
- Export single date images to Google Cloud/Drive.
- Export summary images for each satellite (Sentinel-2 or Landsat series) to Google Cloud/Drive. Summary images are percent of water observations over clear observations.

### 6.1 Add indices

In [ ]:
ls_imcol_bands = ls_imcol.map(iht_functions.addMNDWI).map(iht_functions.addFisher).map(iht_functions.addNDWI)
s_imcol_bands = s_imcol.map(iht_functions.addMNDWI).map(iht_functions.addFisher).map(iht_functions.addNDWI)

### 6.2 Save your pass/fail selections from the graph in Part 5 

In [ ]:
# Get only acceptable passes
passes_only = master_df_reindex[master_df_reindex['sat_result'].str.contains('accept')]  

# Drop any duplicate dates - this won't affect the imagery but should be considered when looking at the gauge data
passes_only = pd.DataFrame(passes_only,columns=['date','Value','sat_result'])
passes_only = passes_only.drop_duplicates('date')

# Keep only dates of interest
dates_of_interest = passes_only['date'].values

# Create array to decide on whether imagery is accepted or rejected
sat_source = passes_only['sat_result'].values

### 6.3 Format file names for outputs

In [ ]:
# Get description to make filenames more useful

# If you have used the extents of the gauge, the gauge name will be used in the file name
if define_gauge:
    description = iht_values.gauge_num.replace(' ', '')
# If you have defined your own coordinates, the gauge, lat, and lon will be used in the file name
elif define_coords:
    # TODO check this works, should have the lats and lons entered in the dashboard
    description = f'{gauge}_lat{iht.input_lat.value}_lon{iht.input_lon.value}'
# If you have uploaded a shapefile, the gauge number and shapefile name will be used in the file name
elif define_shapefile:
    file_ext = shapefile_name.split('/')[-1]
    file_only = file_ext.split('.')[0]
    description = f'{gauge}_{str(file_only)}'

### 6.4 Run the export script

In [ ]:
band_description = iht_values.band_option
output_results = iht_values.export_images

if band_description[0] == 'RGB':
    export_bands = ['red','green','blue']
elif band_description[0] == 'False_colour':
    export_bands = ['swir2','nir','red']
else:
    export_bands = band_description

# Run the export
for i, day in enumerate(dates_of_interest):
    
    date_string = str(passes_only['date'].iloc[i].date())
    
    if sat_source[i] == 'landsat accept':
        date_object = ee.Date.parse('YYYY-MM-dd', date_string)
        end_date = date_object.advance(1,'day')
        ls_im = ls_imcol_bands.filterDate(date_object,end_date).max() # when we do max the si dissapears
        
        for band in export_bands:
            image = ls_im.select(export_bands)
            task = ee.batch.Export.image.toCloudStorage(image=image, # an ee.Image object.
                                                        description=(f'IHT_{description}_landsat_'
                                                                     '{date_string}_{band_description[0]}'),
                                                        bucket = 'mdba_gee_bucket',   
                                                        region=aoi, # an ee.Geometry object.
                                                        scale=30,
                                                        crs='EPSG:4326',
                                                        fileFormat='GeoTIFF')
            if output_results == True:
                task.start()
                print('Exporting',task.status().get('description'))
            else:
                continue

    elif sat_source[i] == 'sentinel accept':
        date_object = ee.Date.parse('YYYY-MM-dd', date_string)
        end_date = date_object.advance(1,'day')
        s_im = s_imcol_bands.filterDate(date_object,end_date).max() # when we do max the si dissapears
        
        for band in export_bands:
            image = s_im.select(export_bands)
            task = ee.batch.Export.image.toCloudStorage(image=image,  # an ee.Image object.
                                                        description=(f'IHT_{description}_sentinel_'
                                                                     f'{date_string}_{band_description[0]}'),
                                                        bucket = 'mdba_gee_bucket',   
                                                        region=aoi, # an ee.Geometry object.
                                                        scale=10,
                                                        crs='EPSG:4326',
                                                        fileFormat='GeoTIFF')
            if output_results == True:
                task.start()
                print('Exporting',task.status().get('description'))
            else:
                continue

#### Check task status

In [ ]:
task.status()

#### Uncomment if required

In [ ]:
# # Kill all tasks in emergency
# for task in ee.batch.Task.list():
#     task.cancel()

## Part 7: Exporting summary images

### 7.1 Calculating summary images

In [ ]:
band = [iht_values.summary_band_option]
water_threshold = iht_values.water_threshold

ls_water_list = []
ls_clear_list = []
s_water_list = []
s_clear_list = []

for i, day in enumerate(dates_of_interest):
        
    date_string = str(passes_only['date'].iloc[i].date())
    
    if sat_source[i] == 'landsat accept':
        date_object = ee.Date.parse('YYYY-MM-dd', date_string)
        end_date = date_object.advance(1,'day')
        ls_im = ls_imcol_bands.filterDate(date_object,end_date).max() # when we do max the si dissapears
        ls_water = ls_im.select(band).gt(water_threshold)
        ls_water_list.append(ls_water)
        ls_clear = ls_im.select('red').gt(0)
        ls_clear_list.append(ls_clear)
    
    elif sat_source[i] == 'sentinel accept':
        date_object = ee.Date.parse('YYYY-MM-dd', date_string)
        end_date = date_object.advance(1,'day')
        s_im = s_imcol_bands.filterDate(date_object,end_date).max() # when we do max the si dissapears
        s_water = s_im.select(band).gt(water_threshold)
        s_water_list.append(s_water)
        s_clear = s_im.select('red').gt(0)
        s_clear_list.append(s_clear)

# Calculate sum of observations of water
ls_sum_water = ee.ImageCollection(ls_water_list).reduce(ee.Reducer.sum()).toInt()
s_sum_water = ee.ImageCollection(s_water_list).reduce(ee.Reducer.sum()).toInt()

# Calculate sum of clear observations
ls_sum_clear = ee.ImageCollection(ls_clear_list).reduce(ee.Reducer.sum()).toInt()
s_sum_clear = ee.ImageCollection(s_clear_list).reduce(ee.Reducer.sum()).toInt()

# Calculate percent water observations of total possible observations
ls_percent = ls_sum_water.divide(ls_sum_clear).multiply(100)
s_percent = s_sum_water.divide(s_sum_clear).multiply(100)

### 7.2 Display summary images on folium map
- Sum_water: showing the maximum extent of water in the date range
- Sum_clear: Showing if there were any pixels covered in cloud for the whole date range
- Percent: Showing the percent of inundation 

In [ ]:
# Create a folium map object.
my_map = folium.Map(location=[-32, 145], zoom_start=5, height=500)

# Set visualization parameters.
vis_params = {'min': 0,'max': 1}
vis_params_percent = {'min': 0,'max': 100, 'palette':['white','0061A1']}

# Add the elevation model to the map object.
try:
    my_map.add_ee_layer(ls_sum_water.clip(aoi), vis_params, 'ls_sum_water')
    my_map.add_ee_layer(ls_sum_clear.clip(aoi), vis_params, 'ls_sum_clear')
    my_map.add_ee_layer(ls_percent.clip(aoi).selfMask(), vis_params_percent, 'ls_percent')
except:
    print('No Landsat layers to add to map')

try:
    my_map.add_ee_layer(s_sum_water.clip(aoi), vis_params, 's_sum_water')
    my_map.add_ee_layer(s_sum_clear.clip(aoi), vis_params, 's_sum_clear')
    my_map.add_ee_layer(s_percent.clip(aoi).selfMask(), vis_params_percent, 's_percent')
except:
    print('No Sentinel layers to add to map')
    
my_map.add_ee_layer(ee.Image(1).clip(aoi), {'palette':['red']}, 'aoi')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
print('Hover over the layers and select the layer of interest to view.')
print('Your area of interest is show in red, click the checkbox to turn it off.')
display(my_map)


### 7.3 Export the summary images

In [ ]:
run_summary = iht_values.export_summary_images

# Run the export
task = ee.batch.Export.image.toCloudStorage(image=s_percent,  # an ee.Image object.
                                            description=f'IHT_{description}_sentinel_percent_{band[0]}',
                                            bucket='mdba_gee_bucket',
                                            region=aoi,  # an ee.Geometry object.
                                            scale=10,
                                            crs='EPSG:4326',
                                            fileFormat='GeoTIFF')
if run_summary:
    task.start()

task = ee.batch.Export.image.toCloudStorage(image=ls_percent,  # an ee.Image object.
                                            description='IHT_{description}_landsat_percent_{band[0]}',
                                            bucket = 'mdba_gee_bucket',
                                            region=aoi,  # an ee.Geometry object.
                                            scale=30,
                                            crs='EPSG:4326',
                                            fileFormat='GeoTIFF')
if run_summary:
    task.start()

#### Check on task status

In [ ]:
task.status()

### Optional outputs 

In [ ]:
# Outputting hydrograph 1: all satellite flyovers on the gauge data:
py.plot(fig_all,filename='hydrograph_outputs/'+description+'_'+datetime.today().strftime('%Y%m%d')+'_all_passes_plot.html')

In [ ]:
# Outputting hydrograph 2: satellite passes divided into rising and falling cats with gauge data
py.plot(graph_rising_falling,filename='hydrograph_outputs/'+description+'_'+datetime.today().strftime('%Y%m%d')+'_rising_falling_plot.html')

In [ ]:
# Outputting hydrograph 3: satellite passes divided into the 
# final accept/reject categories with gauge data:
py.plot(AcceptRejectGraph,filename='hydrograph_outputs/'+description+'_'+datetime.today().strftime('%Y%m%d')+'_accept_reject_plot.html')

In [ ]:
#Exporting the dataframe to the csv
master_df_reindex.to_csv("csv_outputs/"+description+'_'+datetime.today().strftime('%Y%m%d')+'_all_fly_overs.csv')

## IHT complete